In [ ]:
!pip install captum==0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 62.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling n

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd
from captum.attr import FeatureAblation, LLMAttribution, TextTemplateInput
import nltk
import json
import gc
from google.colab import drive
from tqdm.notebook import tqdm, trange
import time
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from typing import Dict, List

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Fri Feb  7 01:28:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             28W /   70W |    2340MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
nltk.download('punkt', quiet=True)
torch.backends.cudnn.benchmark = True

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.backends.cudnn.benchmark = True

### Model Initialization

In [ ]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'left'

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # half precision
    low_cpu_mem_usage=True
).to(device)
model.eval()  # evaluation mode on

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

In [ ]:
plt.style.use('default')
sns.set_theme(style="whitegrid")

### Basic Interaction With The Model

In [ ]:
# def generate_response(prompt, max_length=1000):
#     """Generate response with optimized parameters for GPU processing."""
#     try:
#         messages = [{"role": "user", "content": prompt}]
#         chat = tokenizer.apply_chat_template(messages, tokenize=False)

#         inputs = tokenizer(chat, return_tensors="pt").to(device)

#         with torch.no_grad():
#             outputs = model.generate(
#                 **inputs,
#                 max_length=max_length,
#                 num_return_sequences=1,
#                 temperature=0.7,
#                 do_sample=True,
#                 repetition_penalty=1.2,
#                 pad_token_id=tokenizer.eos_token_id,
#                 early_stopping=False,
#                 no_repeat_ngram_size=3,
#                 num_beams=1,
#                 use_cache=True
#             )

#         response = tokenizer.decode(outputs[0], skip_special_tokens=True)
#         torch.cuda.empty_cache()  # Clear GPU cache
#         return response

#     except Exception as e:
#         print(f"Error in generate_response: {str(e)}")
#         return ""

# test_prompt = "Explain what kitkat is in one sentence."
# print(generate_response(test_prompt))

### Abstract Processing Functions

In [ ]:
def load_abstracts(file_path):
    """Load abstracts from CSV file."""
    df = pd.read_csv(file_path)
    print(f"Loaded {len(df)} abstracts")
    return df

In [ ]:
def split_abstract_into_features(abstract):
    """Split abstract into meaningful features using enhanced classification."""
    try:
        # Download punkt if not already downloaded
        nltk.download('punkt', quiet=True)
        sentences = nltk.sent_tokenize(abstract)
    except LookupError:
        # Fallback to simple sentence splitting if NLTK fails
        sentences = [s.strip() for s in abstract.split('.') if s.strip()]

    if not sentences:
        return {
            'background': [''],
            'objective': [''],
            'methods': [''],
            'results': [''],
            'conclusion': ['']
        }

    # Define section markers
    section_markers = {
        'background': ['background', 'introduction', 'context', 'previously'],
        'objective': ['aim', 'goal', 'objective', 'purpose', 'we propose', 'this paper'],
        'methods': ['method', 'approach', 'technique', 'procedure', 'we develop'],
        'results': ['result', 'find', 'show', 'demonstrate', 'achieve'],
        'conclusion': ['conclude', 'conclusion', 'finally', 'future work', 'implications']
    }

    # Initialize sections
    features = {
        'background': [],
        'objective': [],
        'methods': [],
        'results': [],
        'conclusion': []
    }

    # Classify sentences
    for sentence in sentences:
        sentence_lower = sentence.lower()
        assigned = False

        # Check for section markers
        for section, markers in section_markers.items():
            if any(marker in sentence_lower for marker in markers):
                features[section].append(sentence)
                assigned = True
                break

        # Position-based assignment if no markers found
        if not assigned:
            if len(sentences) <= 3:
                features['methods'].append(sentence)
            else:
                position = sentences.index(sentence) / len(sentences)
                if position < 0.2:
                    features['background'].append(sentence)
                elif position < 0.4:
                    features['objective'].append(sentence)
                elif position < 0.6:
                    features['methods'].append(sentence)
                elif position < 0.8:
                    features['results'].append(sentence)
                else:
                    features['conclusion'].append(sentence)

    return features

###  Analysis Functions

In [ ]:
def generate_response(prompt, max_length=1000):
    """Generate response with optimized parameters for GPU processing."""
    try:
        messages = [{"role": "user", "content": prompt}]
        chat = tokenizer.apply_chat_template(messages, tokenize=False)

        inputs = tokenizer(chat, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_length=max_length,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                repetition_penalty=1.2,
                pad_token_id=tokenizer.eos_token_id,
                early_stopping=False,
                no_repeat_ngram_size=3,
                num_beams=1,
                use_cache=True
            )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        torch.cuda.empty_cache()  # Clear GPU cache
        return response

    except Exception as e:
        print(f"Error in generate_response: {str(e)}")
        return ""

In [ ]:
def analyze_abstract(abstract, title):
    """Analyze a single abstract with progress tracking."""
    try:
        # Generate hypothesis with progress update
        print(f"\rGenerating hypothesis for: {title[:30]}...", end="")
        prompt = f"Read this scientific paper abstract and identify its main hypothesis. Abstract title: {title}\n\nAbstract: {abstract}\n\nWhat is the main hypothesis of this paper? Provide a clear, concise statement."
        hypothesis = generate_response(prompt)

        # Split abstract into features
        print(f"\rSplitting abstract into features...", end="")
        features = split_abstract_into_features(abstract)

        # Set up attribution analysis
        print(f"\rSetting up attribution analysis...", end="")
        fa = FeatureAblation(model)
        llm_attr = LLMAttribution(fa, tokenizer)

        # Create template input using features
        all_sentences = []
        feature_indices = {}
        current_idx = 0

        # Collect all sentences while tracking indices
        for section in ['background', 'objective', 'methods', 'results', 'conclusion']:
            feature_indices[section] = (current_idx, current_idx + len(features[section]))
            all_sentences.extend(features[section])
            current_idx += len(features[section])

        template = " ".join(["{}"] * len(all_sentences))
        inp = TextTemplateInput(template, values=all_sentences)

        # Calculate attribution scores
        print(f"\rCalculating attribution scores...", end="")
        with torch.amp.autocast('cuda'):
            attr_res = llm_attr.attribute(
                inp,
                target=hypothesis,
                skip_tokens=[1]
            )

        # Organize results
        print(f"\rOrganizing results...", end="")
        section_scores = {}
        for section, (start, end) in feature_indices.items():
            section_scores[section] = attr_res.seq_attr[start:end].mean().item() if end > start else 0.0

        # Clear GPU cache
        torch.cuda.empty_cache()

        return {
            'title': title,
            'hypothesis': hypothesis,
            'attribution_scores': section_scores,
            'full_attribution': attr_res.seq_attr.tolist(),
            'features': features
        }

    except Exception as e:
        print(f"\nError analyzing abstract '{title}': {str(e)}")
        return None

### Attribution Analysis Functions

In [ ]:
def calculate_section_metrics(results):
    """Calculate detailed metrics for each section of the abstracts."""
    # Extract all attribution scores
    scores_df = pd.DataFrame([r['attribution_scores'] for r in results])

    metrics = {
        'basic_stats': {
            'mean': scores_df.mean(),
            'median': scores_df.median(),
            'std': scores_df.std(),
            'var': scores_df.var(),
            'skew': scores_df.skew(),
            'kurtosis': scores_df.kurtosis(),
            'min': scores_df.min(),
            'max': scores_df.max(),
        },
        'percentiles': {
            '25th': scores_df.quantile(0.25),
            '50th': scores_df.quantile(0.50),
            '75th': scores_df.quantile(0.75),
            '90th': scores_df.quantile(0.90)
        },
        'relative_importance': scores_df.div(scores_df.sum(axis=1), axis=0).mean(),
        'correlations': scores_df.corr()
    }

    return metrics

def analyze_section_lengths(results):
    """Analyze the distribution of sentences across sections."""
    section_lengths = {
        'background': [],
        'objective': [],
        'methods': [],
        'results': [],
        'conclusion': []
    }

    for result in results:
        for section, sentences in result['features'].items():
            section_lengths[section].append(len(sentences))

    length_df = pd.DataFrame(section_lengths)

    return {
        'mean_lengths': length_df.mean(),
        'std_lengths': length_df.std(),
        'max_lengths': length_df.max(),
        'min_lengths': length_df.min(),
        'distribution': length_df.describe()
    }

def calculate_attribution_stability(results):
    """Calculate stability metrics for attribution scores."""
    scores_df = pd.DataFrame([r['attribution_scores'] for r in results])

    stability_metrics = {
        'coefficient_of_variation': scores_df.std() / scores_df.mean(),
        'iqr': scores_df.quantile(0.75) - scores_df.quantile(0.25),
        'range': scores_df.max() - scores_df.min()
    }

    return stability_metrics

def analyze_section_relationships(results):
    """Analyze relationships between different sections."""
    scores_df = pd.DataFrame([r['attribution_scores'] for r in results])

    relationships = {
        'correlations': scores_df.corr(),
        'covariance': scores_df.cov(),
        'relative_contributions': scores_df.div(scores_df.sum(axis=1), axis=0)
    }

    return relationships

def analyze_trends(results):
    """Enhanced analysis of trends in the attribution scores and features."""
    # Calculate all metrics
    section_metrics = calculate_section_metrics(results)
    length_analysis = analyze_section_lengths(results)
    stability_metrics = calculate_attribution_stability(results)
    section_relationships = analyze_section_relationships(results)

    # Get strongest correlation (excluding self-correlations)
    corr_matrix = section_relationships['correlations']
    # Create mask to exclude diagonal and upper triangle
    mask = np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)
    # Get strongest correlation
    corr_values = corr_matrix.values[mask]
    corr_indices = np.where(mask)
    max_corr_idx = np.argmax(np.abs(corr_values))
    strongest_pair = (
        corr_matrix.index[corr_indices[0][max_corr_idx]],
        corr_matrix.index[corr_indices[1][max_corr_idx]]
    )
    strongest_correlation = corr_values[max_corr_idx]

    # Combine into comprehensive report
    analysis_report = {
        'section_metrics': section_metrics,
        'length_analysis': length_analysis,
        'stability_metrics': stability_metrics,
        'section_relationships': section_relationships,

        # Additional summary statistics
        'summary': {
            'most_important_section': section_metrics['relative_importance'].idxmax(),
            'most_variable_section': section_metrics['basic_stats']['std'].idxmax(),
            'strongest_correlation': {
                'sections': strongest_pair,
                'value': strongest_correlation
            },
            'average_section_length': length_analysis['mean_lengths'].mean()
        }
    }

    return analysis_report

def print_analysis_summary(analysis_report):
    """Print a formatted summary of the analysis results."""
    print("\nANALYSIS SUMMARY")
    print("=" * 50)

    # Basic section statistics
    print("\nSection Attribution Statistics:")
    print("-" * 30)
    for section in analysis_report['section_metrics']['basic_stats']['mean'].index:
        print(f"\n{section.upper()}:")
        print(f"  Mean: {analysis_report['section_metrics']['basic_stats']['mean'][section]:.2f}")
        print(f"  Std Dev: {analysis_report['section_metrics']['basic_stats']['std'][section]:.2f}")
        print(f"  Relative Importance: {analysis_report['section_metrics']['relative_importance'][section]:.2%}")

    # Stability metrics
    print("\nStability Metrics:")
    print("-" * 30)
    for metric, values in analysis_report['stability_metrics'].items():
        print(f"\n{metric}:")
        for section, value in values.items():
            print(f"  {section}: {value:.3f}")

    # Section length analysis
    print("\nSection Length Analysis:")
    print("-" * 30)
    for section, mean_length in analysis_report['length_analysis']['mean_lengths'].items():
        print(f"  {section}: {mean_length:.1f} sentences")

    # Key findings
    print("\nKey Findings:")
    print("-" * 30)
    print(f"Most important section: {analysis_report['summary']['most_important_section']}")
    print(f"Most variable section: {analysis_report['summary']['most_variable_section']}")
    strongest_corr = analysis_report['summary']['strongest_correlation']
    print(f"Strongest correlation: {strongest_corr['sections'][0]} - {strongest_corr['sections'][1]} ({strongest_corr['value']:.3f})")

### Core Visualization Functions

In [ ]:
def create_visualizations(analysis_report: Dict, results: List[Dict], output_path: str = None):
    """Create static visualizations of the analysis results."""
    # Set plotting style
    plt.style.use('default')
    colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd']

    visualizations = []

    # 1. Attribution Score Correlation Heatmap
    fig1 = plt.figure(figsize=(10, 8))
    scores_df = pd.DataFrame([r['attribution_scores'] for r in results])
    sns.heatmap(scores_df.corr(),
                annot=True,
                cmap='RdYlBu_r',
                center=0,
                fmt='.2f')
    plt.title('Attribution Score Correlations Between Sections', pad=20)
    if output_path:
        plt.savefig(f"{output_path}correlation_heatmap.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig1)
    plt.close()

    # 2. Section Importance Distribution
    fig2 = plt.figure(figsize=(10, 8))
    section_importance = analysis_report['section_metrics']['relative_importance']
    bars = plt.bar(range(len(section_importance)),
                  section_importance.values,
                  color=colors[:len(section_importance)])
    plt.xticks(range(len(section_importance)), section_importance.index, rotation=45)
    plt.title('Relative Importance of Each Section', pad=20)
    plt.ylabel('Relative Importance')
    for i, v in enumerate(section_importance.values):
        plt.text(i, v, f'{v:.3f}', ha='center', va='bottom')
    if output_path:
        plt.savefig(f"{output_path}section_importance.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig2)
    plt.close()

    # 3. Section Length Distribution
    fig3 = plt.figure(figsize=(10, 8))
    length_data = pd.DataFrame([{
        section: len(result['features'][section])
        for section in result['features']
    } for result in results])
    sns.boxplot(data=length_data, palette=colors[:len(length_data.columns)])
    plt.title('Distribution of Section Lengths', pad=20)
    plt.xticks(rotation=45)
    plt.ylabel('Number of Sentences')
    if output_path:
        plt.savefig(f"{output_path}length_distribution.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig3)
    plt.close()

    # 4. Stability Metrics Heatmap
    fig4 = plt.figure(figsize=(10, 8))
    stability_df = pd.DataFrame(analysis_report['stability_metrics'])
    sns.heatmap(stability_df,
                annot=True,
                cmap='viridis',
                fmt='.2f')
    plt.title('Stability Metrics Across Sections', pad=20)
    if output_path:
        plt.savefig(f"{output_path}stability_metrics.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig4)
    plt.close()

    # 5. Score Distribution by Section
    fig5 = plt.figure(figsize=(10, 8))
    scores_melted = scores_df.melt(var_name='Section', value_name='Score')
    sns.violinplot(x='Section',
                  y='Score',
                  data=scores_melted,
                  palette='Set2')
    plt.title('Distribution of Attribution Scores by Section', pad=20)
    plt.xticks(rotation=45)
    if output_path:
        plt.savefig(f"{output_path}score_distribution.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig5)
    plt.close()

    # 6. Section Score Trends
    fig6 = plt.figure(figsize=(10, 8))
    mean_scores = scores_df.mean()
    std_scores = scores_df.std()
    x = range(len(mean_scores))
    plt.errorbar(x, mean_scores, yerr=std_scores, fmt='o-', capsize=5)
    plt.xticks(x, mean_scores.index, rotation=45)
    plt.title('Mean Attribution Scores with Standard Deviation', pad=20)
    plt.grid(True)
    if output_path:
        plt.savefig(f"{output_path}score_trends.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig6)
    plt.close()

    # 7. Section Length vs Attribution Score
    fig7 = plt.figure(figsize=(10, 8))
    length_score_corr = pd.DataFrame({
        'length': [len(' '.join(result['features'][section]))
                  for result in results
                  for section in result['features']],
        'score': [score for result in results
                 for score in result['attribution_scores'].values()],
        'section': [section for result in results
                   for section in result['features']]
    })
    sns.scatterplot(data=length_score_corr,
                   x='length',
                   y='score',
                   hue='section',
                   alpha=0.6)
    plt.title('Section Length vs Attribution Score', pad=20)
    plt.xlabel('Section Length (characters)')
    plt.ylabel('Attribution Score')
    if output_path:
        plt.savefig(f"{output_path}length_vs_score.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig7)
    plt.close()

    # 8. Section Variability Analysis
    fig8 = plt.figure(figsize=(10, 8))
    variability_metrics = pd.DataFrame({
        'std': analysis_report['section_metrics']['basic_stats']['std'],
        'iqr': analysis_report['stability_metrics']['iqr'],
        'cv': analysis_report['stability_metrics']['coefficient_of_variation']
    })
    sns.heatmap(variability_metrics.T,
                annot=True,
                cmap='YlOrRd',
                fmt='.2f')
    plt.title('Section Variability Metrics', pad=20)
    if output_path:
        plt.savefig(f"{output_path}variability_analysis.png", dpi=300, bbox_inches='tight')
    visualizations.append(fig8)
    plt.close()

    return visualizations

def generate_visual_analysis(results: List[Dict], analysis_report: Dict, output_path: str = None):
    """Generate static visualizations and return the figures."""
    return create_visualizations(analysis_report, results, output_path)

### Admin

In [ ]:
def save_results(results, output_path):
    """Save results to both JSON and CSV formats."""
    # Save detailed results to JSON
    with open(f"{output_path}.json", 'w') as f:
        json.dump(results, f, indent=2)

    # Create and save summary DataFrame
    summary_df = pd.DataFrame([{
        'title': r['title'],
        'hypothesis': r['hypothesis'],
        'background_score': r['attribution_scores']['background'],
        'objective_score': r['attribution_scores']['objective'],
        'methods_score': r['attribution_scores']['methods'],
        'results_score': r['attribution_scores']['results'],
        'conclusion_score': r['attribution_scores']['conclusion']
    } for r in results])

    summary_df.to_csv(f"{output_path}_summary.csv", index=False)

In [ ]:
def batch_process_abstracts(csv_path, output_path, batch_size=24):
    """Process abstracts in optimized batches with detailed progress tracking."""
    df = load_abstracts(csv_path)
    results = []
    total_abstracts = len(df)

    # Create progress bars
    pbar_batches = tqdm(total=total_abstracts//batch_size + 1,
                       desc='Processing batches',
                       position=0)
    pbar_abstracts = tqdm(total=total_abstracts,
                         desc='Total abstracts processed',
                         position=1)

    # Process in batches
    for i in range(0, total_abstracts, batch_size):
        batch_start_time = time.time()

        # Clear GPU cache at start of each batch
        torch.cuda.empty_cache()

        # Get current batch
        batch = df.iloc[i:i + batch_size]
        batch_results = []

        # Create progress bar for current batch
        pbar_current = tqdm(total=len(batch),
                          desc=f'Current batch ({i//batch_size + 1}/{total_abstracts//batch_size + 1})',
                          position=2,
                          leave=False)

        # Process each abstract in batch
        for _, row in batch.iterrows():
            abstract_start_time = time.time()
            try:
                result = analyze_abstract(row['Abstract'], row['Title'])
                if result is not None:
                    batch_results.append(result)

                # Update progress and timing information
                abstract_time = time.time() - abstract_start_time
                pbar_current.set_postfix({
                    'Title': row['Title'][:20] + '...',
                    'Time': f'{abstract_time:.2f}s'
                })

            except Exception as e:
                print(f"\nError processing abstract '{row['Title']}': {str(e)}")
            finally:
                pbar_current.update(1)
                pbar_abstracts.update(1)

        # Add batch results and save progress
        results.extend(batch_results)
        if len(results) % batch_size == 0:
            save_results(results, output_path)

        # Update batch progress information
        batch_time = time.time() - batch_start_time
        avg_time_per_abstract = batch_time / len(batch)
        estimated_remaining_time = (total_abstracts - (i + len(batch))) * avg_time_per_abstract

        pbar_batches.set_postfix({
            'Batch time': f'{batch_time:.2f}s',
            'Avg time/abstract': f'{avg_time_per_abstract:.2f}s',
            'Est. remaining': f'{estimated_remaining_time/60:.1f}min',
            'GPU Mem': f'{torch.cuda.memory_allocated()/1e9:.1f}GB'
        })
        pbar_batches.update(1)

        # Close current batch progress bar
        pbar_current.close()

    # Close main progress bars
    pbar_batches.close()
    pbar_abstracts.close()

    # Final save
    save_results(results, output_path)
    return results

In [ ]:
# Check if a file exists
file_path = '/content/drive/My Drive/llm_hyp/raw_data_5.csv'
if os.path.isfile(file_path):
    print("File exists")
else:
    print("File does not exist")

# Check if a folder exists
folder_path = r'/content/drive/My Drive/llm_hyp/visuals/'
if os.path.isdir(folder_path):
    print("Folder exists")
else:
    print("Folder does not exist")


File exists
Folder exists


## Main

In [ ]:
# if __name__ == "__main__":

#     results = batch_process_abstracts(
#         csv_path=r'/content/drive/My Drive/llm_hyp/raw_data_2.csv',
#         output_path=r'/content/drive/My Drive/llm_hyp',
#         batch_size=32
#     )

#     analysis_report = analyze_trends(results)
#     print_analysis_summary(analysis_report)
#     fig = generate_visual_analysis(
#         results,
#         analysis_report,
#         output_path=r'/content/drive/My Drive/llm_hyp/visuals/'
#     )

#     plt.show()

In [ ]:
# Run the intensive processing first and save results
#results = batch_process_abstracts(
#    csv_path=r'/content/drive/My Drive/llm_hyp/raw_data_5.csv',
#    output_path=r'/content/drive/My Drive/llm_hyp',
#    batch_size=32
#)

# Save results to a temp file to avoid reprocessing
#temp_results_path = r'/content/drive/My Drive/llm_hyp/temp_results.json'
#with open(temp_results_path, 'w') as f:
#    json.dump(results, f)

In [ ]:
# Load the saved results
temp_results_path = r'/content/drive/My Drive/llm_hyp/temp_results.json'
with open(temp_results_path, 'r') as f:
    results = json.load(f)

# Run analysis and visualization
try:
    # Generate analysis
    analysis_report = analyze_trends(results)
    print_analysis_summary(analysis_report)

    # Generate visualizations
    fig = generate_visual_analysis(
        results,
        analysis_report,
        output_path=r'/content/drive/My Drive/llm_hyp/visuals/'
    )
    plt.show()
except Exception as e:
    print(f"Error in analysis: {str(e)}")


ANALYSIS SUMMARY

Section Attribution Statistics:
------------------------------

BACKGROUND:
  Mean: 59.79
  Std Dev: 19.11
  Relative Importance: 19.73%

OBJECTIVE:
  Mean: 49.49
  Std Dev: 28.86
  Relative Importance: 16.86%

METHODS:
  Mean: 68.30
  Std Dev: 32.30
  Relative Importance: 21.27%

RESULTS:
  Mean: 56.66
  Std Dev: 33.98
  Relative Importance: 18.46%

CONCLUSION:
  Mean: 77.85
  Std Dev: 74.05
  Relative Importance: 23.68%

Stability Metrics:
------------------------------

coefficient_of_variation:
  background: 0.320
  objective: 0.583
  methods: 0.473
  results: 0.600
  conclusion: 0.951

iqr:
  background: 11.064
  objective: 6.219
  methods: 32.599
  results: 1.350
  conclusion: 45.451

range:
  background: 51.806
  objective: 75.530
  methods: 84.584
  results: 92.120
  conclusion: 196.982

Section Length Analysis:
------------------------------
  background: 1.6 sentences
  objective: 2.4 sentences
  methods: 2.0 sentences
  results: 3.0 sentences
  conclusion:

<ipython-input-16-1a769efe6c39>:70: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.violinplot(x='Section',
